# Merging DataFrames

#### import modules

In [1]:
import sys
import pandas as pd

#### version info

In [2]:
print('python {}'.format(sys.version.split(' ')[0]))
print(f'pandas {pd.__version__}')

python 3.6.6
pandas 0.23.4


#### duplicate finder

In [3]:
def duplicates(df, cols):
    grp_size = df.groupby(by=cols).size()
    if not bool((grp_size.values > 1).sum()):
        print('No duplicates found.')
        return None
    else:
        dupl_cnt = grp_size[grp_size.values > 1].rename('aantal')
        dupl_num = dupl_cnt.values.sum() - dupl_cnt.shape[0]
        dupl_pct = dupl_num / df.shape[0] * 100
        print('{} duplicates found in {} rows ({}%).'.format(dupl_num, df.shape[0], round(dupl_pct, 2)))
        return dupl_cnt.sort_values(ascending=False)

#### create dataframes

In [4]:
df1 = pd.DataFrame({'plaats': ['Amsterdam', 'Rotterdam', 'Utrecht'],
                    'inwoners': [805166, 614543, 324723]})
df1

,plaats,inwoners
0,Amsterdam,805166
1,Rotterdam,614543
2,Utrecht,324723


In [5]:
df2 = pd.DataFrame({'plaats': ['Rotterdam', 'Utrecht', 'Leeuwarden', 'Maastricht'],
                    'provincie': ['Zuid-Holland', 'Utrecht', 'Friesland', 'Limburg']})
df2

,plaats,provincie
0,Rotterdam,Zuid-Holland
1,Utrecht,Utrecht
2,Leeuwarden,Friesland
3,Maastricht,Limburg


In [6]:
duplicates(df1, 'plaats')

No duplicates found.


In [7]:
duplicates(df2, 'plaats')

No duplicates found.


#### merges

In [8]:
# default
pd.merge(df1, df2)

,plaats,inwoners,provincie
0,Rotterdam,614543,Zuid-Holland
1,Utrecht,324723,Utrecht


In [9]:
# inner
pd.merge(df1, df2, how='inner', on='plaats')

,plaats,inwoners,provincie
0,Rotterdam,614543,Zuid-Holland
1,Utrecht,324723,Utrecht


In [10]:
# left
pd.merge(df1, df2, how='left', on='plaats')

,plaats,inwoners,provincie
0,Amsterdam,805166,NaN
1,Rotterdam,614543,Zuid-Holland
2,Utrecht,324723,Utrecht


In [11]:
# right
pd.merge(df1, df2, how='right', on='plaats')

,plaats,inwoners,provincie
0,Rotterdam,614543.0,Zuid-Holland
1,Utrecht,324723.0,Utrecht
2,Leeuwarden,NaN,Friesland
3,Maastricht,NaN,Limburg


In [12]:
# outer
pd.merge(df1, df2, how='outer', on='plaats')

,plaats,inwoners,provincie
0,Amsterdam,805166.0,NaN
1,Rotterdam,614543.0,Zuid-Holland
2,Utrecht,324723.0,Utrecht
3,Leeuwarden,NaN,Friesland
4,Maastricht,NaN,Limburg


In [13]:
# outer + indicator
pd.merge(df1, df2, how='outer', on='plaats', indicator=True)

,plaats,inwoners,provincie,_merge
0,Amsterdam,805166.0,NaN,left_only
1,Rotterdam,614543.0,Zuid-Holland,both
2,Utrecht,324723.0,Utrecht,both
3,Leeuwarden,NaN,Friesland,right_only
4,Maastricht,NaN,Limburg,right_only


#### add rows

In [14]:
df1 = df1.append([{'plaats': 'Bergen', 'inwoners': 29913},
                  {'plaats': 'Bergen', 'inwoners': 95299}], ignore_index=True)
df1

,plaats,inwoners
0,Amsterdam,805166
1,Rotterdam,614543
2,Utrecht,324723
3,Bergen,29913
4,Bergen,95299


In [15]:
df2 = df2.append([{'plaats': 'Bergen', 'provincie': 'Noord-Holland'},
                  {'plaats': 'Bergen', 'provincie': 'Henegouwen'}], ignore_index=True)
df2

,plaats,provincie
0,Rotterdam,Zuid-Holland
1,Utrecht,Utrecht
2,Leeuwarden,Friesland
3,Maastricht,Limburg
4,Bergen,Noord-Holland
5,Bergen,Henegouwen


In [16]:
duplicates(df1, 'plaats')

1 duplicates found in 5 rows (20.0%).


plaats
Bergen    2
Name: aantal, dtype: int64

In [17]:
duplicates(df2, 'plaats')

1 duplicates found in 6 rows (16.67%).


plaats
Bergen    2
Name: aantal, dtype: int64

#### merge

In [18]:
pd.merge(df1, df2, how='left', on='plaats')

,plaats,inwoners,provincie
0,Amsterdam,805166,NaN
1,Rotterdam,614543,Zuid-Holland
2,Utrecht,324723,Utrecht
3,Bergen,29913,Noord-Holland
4,Bergen,29913,Henegouwen
5,Bergen,95299,Noord-Holland
6,Bergen,95299,Henegouwen


#### add columns

In [19]:
df1['land'] = ['Nederland', 'Nederland', 'Nederland', 'Nederland', 'België']
df1

,plaats,inwoners,land
0,Amsterdam,805166,Nederland
1,Rotterdam,614543,Nederland
2,Utrecht,324723,Nederland
3,Bergen,29913,Nederland
4,Bergen,95299,België


In [20]:
df2['land'] = ['Nederland', 'Nederland', 'Nederland', 'Nederland', 'Nederland', 'België']
df2

,plaats,provincie,land
0,Rotterdam,Zuid-Holland,Nederland
1,Utrecht,Utrecht,Nederland
2,Leeuwarden,Friesland,Nederland
3,Maastricht,Limburg,Nederland
4,Bergen,Noord-Holland,Nederland
5,Bergen,Henegouwen,België


In [21]:
duplicates(df1, ['plaats', 'land'])

No duplicates found.


In [22]:
duplicates(df2, ['plaats', 'land'])

No duplicates found.


#### merge

In [23]:
pd.merge(df1, df2)

,plaats,inwoners,land,provincie
0,Rotterdam,614543,Nederland,Zuid-Holland
1,Utrecht,324723,Nederland,Utrecht
2,Bergen,29913,Nederland,Noord-Holland
3,Bergen,95299,België,Henegouwen


In [24]:
pd.merge(df1, df2, how='inner', on=['plaats', 'land'])

,plaats,inwoners,land,provincie
0,Rotterdam,614543,Nederland,Zuid-Holland
1,Utrecht,324723,Nederland,Utrecht
2,Bergen,29913,Nederland,Noord-Holland
3,Bergen,95299,België,Henegouwen


#### add duplicates

In [25]:
df3 = df1.copy()
df3 = df3.append([df3.iloc[-1, :],
                  df3.iloc[-1, :],
                  df3.iloc[0, :],
                  df3.iloc[1, :]]).reset_index(drop=True)
df3

,plaats,inwoners,land
0,Amsterdam,805166,Nederland
1,Rotterdam,614543,Nederland
2,Utrecht,324723,Nederland
3,Bergen,29913,Nederland
4,Bergen,95299,België
5,Bergen,95299,België
6,Bergen,95299,België
7,Amsterdam,805166,Nederland
8,Rotterdam,614543,Nederland


In [26]:
duplicates(df3, ['plaats', 'land'])

4 duplicates found in 9 rows (44.44%).


plaats     land     
Bergen     België       3
Rotterdam  Nederland    2
Amsterdam  Nederland    2
Name: aantal, dtype: int64

In [27]:
duplicates(df3, ['plaats', 'land']).to_frame()

4 duplicates found in 9 rows (44.44%).


,,aantal
plaats,land,
Bergen,België,3
Rotterdam,Nederland,2
Amsterdam,Nederland,2


#### rename column

In [28]:
df2.rename(columns={'plaats':'stad'}, inplace=True)

In [29]:
df1

,plaats,inwoners,land
0,Amsterdam,805166,Nederland
1,Rotterdam,614543,Nederland
2,Utrecht,324723,Nederland
3,Bergen,29913,Nederland
4,Bergen,95299,België


In [30]:
df2

,stad,provincie,land
0,Rotterdam,Zuid-Holland,Nederland
1,Utrecht,Utrecht,Nederland
2,Leeuwarden,Friesland,Nederland
3,Maastricht,Limburg,Nederland
4,Bergen,Noord-Holland,Nederland
5,Bergen,Henegouwen,België


#### merges

In [31]:
# left_on, right_on
pd.merge(df1, df2, how='inner',
         left_on =['plaats', 'land'],
         right_on=['stad'  , 'land'])

,plaats,inwoners,land,stad,provincie
0,Rotterdam,614543,Nederland,Rotterdam,Zuid-Holland
1,Utrecht,324723,Nederland,Utrecht,Utrecht
2,Bergen,29913,Nederland,Bergen,Noord-Holland
3,Bergen,95299,België,Bergen,Henegouwen


In [32]:
# drop column
pd.merge(df1, df2, how='inner',
         left_on =['plaats', 'land'],
         right_on=['stad'  , 'land']).drop(['stad'], axis=1)

,plaats,inwoners,land,provincie
0,Rotterdam,614543,Nederland,Zuid-Holland
1,Utrecht,324723,Nederland,Utrecht
2,Bergen,29913,Nederland,Noord-Holland
3,Bergen,95299,België,Henegouwen
